In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from torch.utils.data import WeightedRandomSampler
from PIL import Image

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix


# Etap 2
Z ostatniego etapu otrzymaliśmy zdjęcia przygotowane do następnego etapu i zapisane w foldderze 'output'.
Budując i testując modele musimy zwrócić uwagę na brak zbalansowania zbioru. Będziemy się starać balansować zbiór, tak aby Accuracy wykorzystywane do stopnia wytrenowania modelu było adekwatną miarą oceny.

In [2]:
# https://discuss.pytorch.org/t/balanced-sampling-between-classes-with-torchvision-dataloader/2703/2

class ImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform 
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])  
        label = torch.tensor(self.labels[idx], dtype=torch.long) 

        if self.transform:
            image = self.transform(image)
            
        
        return image, label

## Funckcje do treningu i ewaluacji

In [3]:
def train_and_evaluate(model, train_loader, test_loader, epochs=10, learning_rate=0.001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    print(f"Using device: {device}") 
    model.to(device)
  
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    print('PRZED EPOKAMI')
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        model.train()
        print('po model.train')
        running_loss = 0.0
        correct, total = 0, 0
        print('po zerowaniu poprawnych i całkowitych')
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")
    print('po epokach')
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_acc = 100 * correct / total
    print(f"Test Accuracy: {test_acc:.2f}%")
    return model


In [4]:
def train_and_evaluate_more_metrics(model, train_loader, test_loader, epochs=10, learning_rate=0.001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    print(f"Using device: {device}") 
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        all_labels = []
        all_predictions = []
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_labels.extend(labels.cpu().numpy())  
            all_predictions.extend(predicted.cpu().numpy())  
        
        train_acc = 100 * correct / total
        train_precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
        train_recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
        train_f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}")
    
    # Evaluate on test set
    model.eval()
    correct, total = 0, 0
    all_labels = []
    all_predictions = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    test_acc = 100 * correct / total
    test_precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
    test_recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
    test_f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)
    test_conf_matrix = confusion_matrix(all_labels, all_predictions)

    print(f"Test Accuracy: {test_acc:.2f}%, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")
    print("Confusion Matrix:\n", test_conf_matrix)

    return model


In [5]:
def save_to_list(output_root):
    X = []
    y = []
    label_to_index = {
        label: idx for idx, label in enumerate({'AVM', 'Normal', 'Ulcer'})
    }
    print(label_to_index)
    for root, _, files in os.walk(output_root):
        for file in files:
            if file.endswith(".bmp"): 
                input_path = os.path.join(root, file)

                base, ext = os.path.splitext(file)
                new_filename = f"{base}{ext}"
                image = cv2.imread(input_path)

                X.append(image)
                y.append(label_to_index[new_filename.split("_")[0]])
    return X, y

## Pierwsze podejście
Do tej grupy modelów balansowanie zbiorów uzyskaliśmy za pomocą obrotów i odbić i ponownego zapisu plików do folderu 'output_balanced'. Ze względu na konieczność przechowywania dużej liczby zdjęć nie jest to najlepsze wyjście, ale jest to nasz punkt startowy.

### Balansowanie - Oversampling

Funkcja balansująca klasy. Wynik folder 'output_balanced'

In [6]:

input_root = "output"
output_root = "output_balanced"

augmentation_times = {
    "Normal": ["h", "v"],  # 2 razy więcej 
    "AVM": ["h90", "h180", "h270", "v90", "v180", "v270"],  # 6 razy więcej
    "Ulcer": ["h", "h90", "h180", "h270", "v", "v90", "v180", "v270"],  # 8 razy więcej 
}

for root, _, files in os.walk(input_root):
    class_name = os.path.basename(root)  

    if class_name not in augmentation_times:
        continue 

    for file in files:
        if not file.endswith(".bmp"):
            continue 

        input_path = os.path.join(root, file)

        relative_path = os.path.relpath(root, input_root)
        output_dir = os.path.join(output_root, relative_path)
        os.makedirs(output_dir, exist_ok=True)

        base, ext = os.path.splitext(file)
        variants = augmentation_times[class_name]

        image = cv2.imread(input_path)

        transformed_images = {
            "base" : image,
            "h": cv2.flip(image, 1),
            "v": cv2.flip(image, 0),
            "90": cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE),
            "180": cv2.rotate(image, cv2.ROTATE_180),
            "270": cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE),
            "h90": cv2.flip(cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE), 1),
            "h180": cv2.flip(cv2.rotate(image, cv2.ROTATE_180), 1),
            "h270": cv2.flip(cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE), 1),
            "v90": cv2.flip(cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE), 0),
            "v180": cv2.flip(cv2.rotate(image, cv2.ROTATE_180), 0),
            "v270": cv2.flip(cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE), 0)
        }

        # Zapisujemy tylko te transformacje, które są w augmentation_config dla danej klasy
        for variant in variants:
            output_path = os.path.join(output_dir, f"{base}{variant}{ext}")
            cv2.imwrite(output_path, transformed_images[variant])


Sprawdzenie rozkładu klas po balansowaniu.

In [6]:
parent_folder = "output_balanced"
output_data = []  

for folder in os.listdir(parent_folder):
    folder_path = os.path.join(parent_folder, folder)
    if os.path.isdir(folder_path):
        count = sum(1 for f in os.listdir(folder_path) if f.endswith(".bmp"))
        output_data.append([folder, count])  
df_output = pd.DataFrame(output_data, columns=["Nazwa klasy", "Liczba zdjęć"])
df_output

,Nazwa klasy,Liczba zdjęć
0,AVM,4008
1,Normal,4304
2,Ulcer,3720


Procent zbalansowania: 

In [7]:
proc_balanced = df_output.loc[2, "Liczba zdjęć"] /df_output.loc[1, "Liczba zdjęć"] *100
print(f"Procent zbalansowania: {proc_balanced:.2f}%")

Procent zbalansowania: 86.43%


### Podział dataset

Stratify w celu zachowania równowagi pomiędzy zbiorami.

In [8]:
X, y = save_to_list("output_balanced")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=123, stratify=y_test)

{'Ulcer': 0, 'AVM': 1, 'Normal': 2}


### Dataset Preparation 1

Dla zbioru treningowego dodajemy rotacje odpicia w celu większego urozmaicenia zdjęć (augumentacja). Dla zbioru testowego tylko zmiana rozmiaru i normalizacja.

In [9]:

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),          
    transforms.RandomRotation((-30,30)),  
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])  


train_dataset = ImageDataset(X_train, y_train, transform=train_transform)
test_dataset = ImageDataset(X_test, y_test, transform=test_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True)

### Model 1

nn.Droput(0.5) <- zapobiega overfittingowi

In [10]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=3):
        super(CNNModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),  # ewentualnie dodać jeszcze jedną warstwę pośrednią
            nn.ReLU(),
            nn.Dropout(0.5), # zapobieganie overfittingowi
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [11]:
model_1 = CNNModel(num_classes=3)
trained_model_1 = train_and_evaluate_more_metrics(model_1, train_loader, test_loader, epochs=15)
torch.save(trained_model_1.state_dict(), "models_output/model1.pth")

Using device: cuda
Epoch 1/15, Loss: 0.8582, Train Acc: 60.57%, Precision: 0.6136, Recall: 0.6057, F1: 0.6051
Epoch 2/15, Loss: 0.5900, Train Acc: 76.80%, Precision: 0.7727, Recall: 0.7680, F1: 0.7687
Epoch 3/15, Loss: 0.4246, Train Acc: 84.35%, Precision: 0.8467, Recall: 0.8435, F1: 0.8438
Epoch 4/15, Loss: 0.3560, Train Acc: 87.01%, Precision: 0.8734, Recall: 0.8701, F1: 0.8703
Epoch 5/15, Loss: 0.3425, Train Acc: 87.01%, Precision: 0.8730, Recall: 0.8701, F1: 0.8705
Epoch 6/15, Loss: 0.2964, Train Acc: 88.98%, Precision: 0.8929, Recall: 0.8898, F1: 0.8902
Epoch 7/15, Loss: 0.2696, Train Acc: 89.85%, Precision: 0.9011, Recall: 0.8985, F1: 0.8987
Epoch 8/15, Loss: 0.2383, Train Acc: 90.89%, Precision: 0.9111, Recall: 0.9089, F1: 0.9091
Epoch 9/15, Loss: 0.2323, Train Acc: 91.14%, Precision: 0.9137, Recall: 0.9114, F1: 0.9117
Epoch 10/15, Loss: 0.2186, Train Acc: 91.83%, Precision: 0.9198, Recall: 0.9183, F1: 0.9185
Epoch 11/15, Loss: 0.2039, Train Acc: 92.21%, Precision: 0.9236, Recal

Model 65 razy pomylił inne klasy jako Normal

Dodanie innych metryk nie wniosło nowych informacji poza tym, że dane są dobrze zbalansowane oraz model dobrze generalizuje.
Interpretacja ConfusionMatrix: Na przekątnej wartości dobrze zidentyfikowane.

### Model 2

Model zmienił funkcję aktywacji z ReLU na LeakyReLU, która przepuszcza wartośći ujemne w pewnym stopniu (ustawiony na 0.1).
ReLU : jeśli otrzyma dużą liczbę ujmenych wartości to gradient może zatrzymać się na 0 i przestać się uczyć.
LeakyReLU: jesli otrzyma dużą liczbę ujmenych wartość to gradient nie będzie 0 tylko mały i proces uczenia nie zatrzyma się.

In [12]:
class CNNModel_gradient(nn.Module):
    def __init__(self, num_classes=3):
        super(CNNModel_gradient, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1),  \
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1),  
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1),  
            nn.MaxPool2d(2, 2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),  
            nn.ReLU(),
            nn.Dropout(0.5), # zapobieganie overfittingowi
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [13]:
model2 = CNNModel_gradient(num_classes=3)
trained_model2 = train_and_evaluate_more_metrics(model2, train_loader, test_loader, epochs=15)
torch.save(trained_model2.state_dict(), "models_output/model2.pth")

Using device: cuda
Epoch 1/15, Loss: 0.9003, Train Acc: 59.24%, Precision: 0.6072, Recall: 0.5924, F1: 0.5945
Epoch 2/15, Loss: 0.6314, Train Acc: 74.73%, Precision: 0.7520, Recall: 0.7473, F1: 0.7482
Epoch 3/15, Loss: 0.4641, Train Acc: 82.83%, Precision: 0.8304, Recall: 0.8283, F1: 0.8285
Epoch 4/15, Loss: 0.3848, Train Acc: 85.47%, Precision: 0.8569, Recall: 0.8547, F1: 0.8549
Epoch 5/15, Loss: 0.3880, Train Acc: 85.35%, Precision: 0.8561, Recall: 0.8535, F1: 0.8537
Epoch 6/15, Loss: 0.3433, Train Acc: 87.32%, Precision: 0.8760, Recall: 0.8732, F1: 0.8735
Epoch 7/15, Loss: 0.3057, Train Acc: 88.48%, Precision: 0.8866, Recall: 0.8848, F1: 0.8850
Epoch 8/15, Loss: 0.3159, Train Acc: 88.14%, Precision: 0.8839, Recall: 0.8814, F1: 0.8816
Epoch 9/15, Loss: 0.3192, Train Acc: 88.27%, Precision: 0.8849, Recall: 0.8827, F1: 0.8829
Epoch 10/15, Loss: 0.2536, Train Acc: 90.41%, Precision: 0.9058, Recall: 0.9041, F1: 0.9043
Epoch 11/15, Loss: 0.2781, Train Acc: 89.60%, Precision: 0.8978, Recal

LeakyReLU może powodować, że gradienty są mniejsze dla wartości ujemnych, co może spowolnić uczenie się w pewnych przypadkach. -> widzimy spadek accuracy. 

### Model 3

Zmiana nn.Dropout na 0.2 -> sprawdzenie czy w przypadku 1 modelu nie zachodizło zjawisko underfittingu.

In [14]:
class CNNModel_lower_dropout(nn.Module):
    def __init__(self, num_classes=3):
        super(CNNModel_lower_dropout, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),  # ewentualnie dodać jeszcze jedną warstwę pośrednią
            nn.ReLU(),
            nn.Dropout(0.2), # zapobieganie overfittingowi
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [15]:
model3 = CNNModel_lower_dropout(num_classes=3)
trained_model3 = train_and_evaluate_more_metrics(model3, train_loader, test_loader, epochs=15)
torch.save(trained_model3.state_dict(), "models_output/model3.pth")

Using device: cuda
Epoch 1/15, Loss: 0.9589, Train Acc: 60.28%, Precision: 0.6126, Recall: 0.6028, F1: 0.6042
Epoch 2/15, Loss: 0.6386, Train Acc: 73.76%, Precision: 0.7424, Recall: 0.7376, F1: 0.7389
Epoch 3/15, Loss: 0.5358, Train Acc: 79.52%, Precision: 0.7975, Recall: 0.7952, F1: 0.7955
Epoch 4/15, Loss: 0.4485, Train Acc: 83.06%, Precision: 0.8331, Recall: 0.8306, F1: 0.8309
Epoch 5/15, Loss: 0.4163, Train Acc: 83.85%, Precision: 0.8412, Recall: 0.8385, F1: 0.8389
Epoch 6/15, Loss: 0.3564, Train Acc: 86.61%, Precision: 0.8679, Recall: 0.8661, F1: 0.8663
Epoch 7/15, Loss: 0.3125, Train Acc: 88.27%, Precision: 0.8844, Recall: 0.8827, F1: 0.8829
Epoch 8/15, Loss: 0.2734, Train Acc: 89.67%, Precision: 0.8989, Recall: 0.8967, F1: 0.8969
Epoch 9/15, Loss: 0.2622, Train Acc: 90.00%, Precision: 0.9011, Recall: 0.9000, F1: 0.9001
Epoch 10/15, Loss: 0.2211, Train Acc: 91.33%, Precision: 0.9145, Recall: 0.9133, F1: 0.9135
Epoch 11/15, Loss: 0.2190, Train Acc: 91.77%, Precision: 0.9190, Recal

### Model 4

Zwiększenie ilości filtrów. dla małego zbiru danych może nie być najlepsze, ponieważ model może zacząć się uczyć na pamięć.

In [16]:
class CNNModel_more_filters(nn.Module):
    def __init__(self, num_classes=3):
        super(CNNModel_more_filters, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 28 * 28, 512),  
            # -> tu ustawiamy liczbę filtrów w ostatniej warstwie konwolucyjnej 
            # czyli Conv2d razy wymiary po wszystkich maxPoolach
            nn.ReLU(),
            nn.Dropout(0.5), # zapobieganie overfittingowi
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [17]:
model4 = CNNModel_more_filters(num_classes=3)
trained_model4 = train_and_evaluate_more_metrics(model4, train_loader, test_loader, epochs=15)
torch.save(trained_model4.state_dict(), "models_output/model4.pth")

Using device: cuda
Epoch 1/15, Loss: 1.0083, Train Acc: 59.67%, Precision: 0.6128, Recall: 0.5967, F1: 0.5972
Epoch 2/15, Loss: 0.7237, Train Acc: 69.63%, Precision: 0.7038, Recall: 0.6963, F1: 0.6979
Epoch 3/15, Loss: 0.5551, Train Acc: 78.18%, Precision: 0.7844, Recall: 0.7818, F1: 0.7821
Epoch 4/15, Loss: 0.4604, Train Acc: 82.69%, Precision: 0.8286, Recall: 0.8269, F1: 0.8270
Epoch 5/15, Loss: 0.3988, Train Acc: 85.36%, Precision: 0.8557, Recall: 0.8536, F1: 0.8537
Epoch 6/15, Loss: 0.3462, Train Acc: 87.02%, Precision: 0.8725, Recall: 0.8702, F1: 0.8705
Epoch 7/15, Loss: 0.3159, Train Acc: 88.30%, Precision: 0.8855, Recall: 0.8830, F1: 0.8833
Epoch 8/15, Loss: 0.2775, Train Acc: 89.75%, Precision: 0.8999, Recall: 0.8975, F1: 0.8978
Epoch 9/15, Loss: 0.2421, Train Acc: 90.74%, Precision: 0.9092, Recall: 0.9074, F1: 0.9075
Epoch 10/15, Loss: 0.2564, Train Acc: 90.56%, Precision: 0.9078, Recall: 0.9056, F1: 0.9058
Epoch 11/15, Loss: 0.2535, Train Acc: 90.67%, Precision: 0.9081, Recal

### Model 5

Zwiększenie warstw o jeden.

In [18]:
class CNNModel_more_layers(nn.Module):
    def __init__(self, num_classes=3):
        super(CNNModel_more_layers, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # Nowa warstwa MaxPooling
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 512),  
            nn.ReLU(),
            nn.Dropout(0.5), # zapobieganie overfittingowi
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x



In [19]:
model5 = CNNModel_more_layers(num_classes=3)
trained_model5 = train_and_evaluate_more_metrics(model5, train_loader, test_loader, epochs=15)
torch.save(trained_model5.state_dict(), "models_output/model5.pth")

Using device: cuda
Epoch 1/15, Loss: 0.9154, Train Acc: 56.61%, Precision: 0.5772, Recall: 0.5661, F1: 0.5624
Epoch 2/15, Loss: 0.6970, Train Acc: 70.60%, Precision: 0.7139, Recall: 0.7060, F1: 0.7081
Epoch 3/15, Loss: 0.5687, Train Acc: 77.65%, Precision: 0.7800, Recall: 0.7765, F1: 0.7771
Epoch 4/15, Loss: 0.3980, Train Acc: 85.22%, Precision: 0.8541, Recall: 0.8522, F1: 0.8524
Epoch 5/15, Loss: 0.3187, Train Acc: 88.06%, Precision: 0.8831, Recall: 0.8806, F1: 0.8809
Epoch 6/15, Loss: 0.2889, Train Acc: 89.24%, Precision: 0.8947, Recall: 0.8924, F1: 0.8927
Epoch 7/15, Loss: 0.2389, Train Acc: 90.77%, Precision: 0.9095, Recall: 0.9077, F1: 0.9079
Epoch 8/15, Loss: 0.2262, Train Acc: 91.63%, Precision: 0.9176, Recall: 0.9163, F1: 0.9164
Epoch 9/15, Loss: 0.2038, Train Acc: 92.53%, Precision: 0.9264, Recall: 0.9253, F1: 0.9254
Epoch 10/15, Loss: 0.1894, Train Acc: 92.46%, Precision: 0.9253, Recall: 0.9246, F1: 0.9247
Epoch 11/15, Loss: 0.1957, Train Acc: 92.69%, Precision: 0.9279, Recal

### Model 6
W stosunku do modelu 5 zmniejszono dropout do 0.3.

In [20]:
class CNNModel_more_layers_lower_dropout(nn.Module):
    def __init__(self, num_classes=3):
        super(CNNModel_more_layers_lower_dropout, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2, 2) 
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 512), 
            nn.ReLU(),
            nn.Dropout(0.3), 
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [21]:
model6 = CNNModel_more_layers_lower_dropout(num_classes=3)
trained_model6 = train_and_evaluate_more_metrics(model6, train_loader, test_loader, epochs=15)
torch.save(trained_model6.state_dict(), "models_output/model6.pth")

Using device: cuda
Epoch 1/15, Loss: 0.8967, Train Acc: 58.85%, Precision: 0.5997, Recall: 0.5885, F1: 0.5871
Epoch 2/15, Loss: 0.6860, Train Acc: 71.34%, Precision: 0.7197, Recall: 0.7134, F1: 0.7149
Epoch 3/15, Loss: 0.4715, Train Acc: 82.45%, Precision: 0.8263, Recall: 0.8245, F1: 0.8246
Epoch 4/15, Loss: 0.3922, Train Acc: 85.34%, Precision: 0.8556, Recall: 0.8534, F1: 0.8535
Epoch 5/15, Loss: 0.3171, Train Acc: 88.54%, Precision: 0.8875, Recall: 0.8854, F1: 0.8855
Epoch 6/15, Loss: 0.2886, Train Acc: 89.04%, Precision: 0.8927, Recall: 0.8904, F1: 0.8906
Epoch 7/15, Loss: 0.2808, Train Acc: 89.60%, Precision: 0.8979, Recall: 0.8960, F1: 0.8961
Epoch 8/15, Loss: 0.2331, Train Acc: 91.19%, Precision: 0.9134, Recall: 0.9119, F1: 0.9121
Epoch 9/15, Loss: 0.2175, Train Acc: 91.95%, Precision: 0.9208, Recall: 0.9195, F1: 0.9196
Epoch 10/15, Loss: 0.2000, Train Acc: 92.56%, Precision: 0.9268, Recall: 0.9256, F1: 0.9257
Epoch 11/15, Loss: 0.1883, Train Acc: 93.07%, Precision: 0.9317, Recal

Najlepsze acc = 95.56% -> model 6.

# Inne podejście do balansowania

In [6]:
X, y = save_to_list("output")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=123, stratify=y_test)

{'Ulcer': 0, 'AVM': 1, 'Normal': 2}


# Wczytanie modeli

In [ ]:
model = CNNModel(num_classes=3)  
model.load_state_dict(torch.load("model.pth"))
model.eval()